In [ ]:
import camelot
import numpy as np
import pandas as pd
import json
import os
#import pdb

df_str_cc_lmt,df_str_due_dt,df_str_cc_stmt,concat_df_lmt,concat_df_datconcat_df_smt = '','','','',''
df_cc_lmt, df_due_dt, df_cc_stmt = [],[],[]

pd.options.mode.chained_assignment = None

In [ ]:
with open('config.json') as f:
    config = json.load(f)

in_file = config['cc_pdf_input_location']
password = config['cc_password']
file_list = [x.split('.')[0] for x in os.listdir(in_file['cc_pdf_input_location'])]
out_file = config['cc_csv_output_location']

for i in range(len(file_list)):
    df_cc_lmt.append(str("df_cc_lmt{0}".format(i)))
    df_str_cc_lmt += """df_cc_lmt[{0}],""".format(i)

    df_due_dt.append(str("df_due_dt{0}".format(i)))
    df_str_due_dt += """df_due_dt[{0}],""".format(i)

    df_cc_stmt.append(str("df_cc_stmt{0}".format(i)))
    df_str_cc_stmt += """df_cc_stmt[{0}],""".format(i)

concat_df_lmt = """concatenated_df = pd.concat(["""+df_str_cc_lmt[:-1]+"""],ignore_index=True,axis = 0)"""
concat_df_dat = """concatenated_df = pd.concat(["""+df_str_due_dt[:-1]+"""],ignore_index=True,axis = 0)"""
concat_df_smt = """concatenated_df = pd.concat(["""+df_str_cc_stmt[:-1]+"""],ignore_index=True,axis = 0)"""

In [ ]:
def cc_stmt_decode(pdf_path,out_path,password):
    global pdf_due_df,pdf_cc_limit_df,pdf_cc_stmt_df
    cc_stmt_lst = []

    tables = camelot.read_pdf(pdf_path,pages='all',password=password)

    pdf_due_df = pd.DataFrame(tables[0].df.iloc[1:3])
    pdf_due_df = pdf_due_df.rename(columns=pdf_due_df.iloc[0]).drop(pdf_due_df.index[0])

    pdf_cc_limit_df = pd.DataFrame(tables[0].df.iloc[3:])
    pdf_cc_limit_df = pdf_cc_limit_df.rename(columns=pdf_cc_limit_df.iloc[0]).drop(pdf_cc_limit_df.index[0])

    pdf_cc_stmt_df = pd.DataFrame(tables[1].df.iloc[0:])
    pdf_cc_stmt_df = pdf_cc_stmt_df[pdf_cc_stmt_df[0] != 'Domestic Transactions']
    pdf_cc_stmt_df = pdf_cc_stmt_df.rename(columns=pdf_cc_stmt_df.iloc[0]).drop(pdf_cc_stmt_df.index[0])

    cc_stmt_lst_dict = pdf_cc_stmt_df.to_dict()
    if 'Feature Reward\nPoints' in cc_stmt_lst_dict.keys():
        del cc_stmt_lst_dict['Feature Reward\nPoints']
    dic_key = list(cc_stmt_lst_dict.keys())

    for i in cc_stmt_lst_dict:
        for j in cc_stmt_lst_dict[i]:
            cc_stmt_lst.append(str(cc_stmt_lst_dict[i][j]).split('\n'))

    cc_stmt_lst = [[x for x in sub if x != ' ' and x != 'DHEERAJ PATEL' and x!= 'RUPAL  PATIDAR'] for sub in cc_stmt_lst]
    pdf_cc_stmt_df = pd.DataFrame({dic_key[0]:cc_stmt_lst[0],dic_key[1]:cc_stmt_lst[1],dic_key[2]:cc_stmt_lst[2]})

    with pd.ExcelWriter(out_path) as writer:
        pdf_due_df.to_excel(writer, sheet_name='CC due date')  
        pdf_cc_limit_df.to_excel(writer, sheet_name='CC limit')
        pdf_cc_stmt_df.to_excel(writer, sheet_name='CC stmt')

    return pdf_due_df,pdf_cc_limit_df,pdf_cc_stmt_df

In [ ]:
for i in range(len(file_list)):
    print('File number: ',i+1)
    print('**In-put file {0} read and conversion to xlsx started**'.format(in_file['cc_pdf_input_location']+file_list[i]+'.pdf'))
    df_due_dt[i],df_cc_lmt[i],df_cc_stmt[i] = cc_stmt_decode(in_file['cc_pdf_input_location']+file_list[i]+'.pdf',out_file['cc_csv_output_location']+file_list[i]+'.xlsx',password['cc_password'])
    #df[i] = pd.DataFrame(pdf_cc_stmt_df)
    print('**Output file {0} created**'.format(out_file['cc_csv_output_location']+file_list[i]+'.xlsx'))
    print(' ')
#print(concat_df_lmt,concat_df_dat,concat_df_smt)